A *degree n pillowcase-tiled surface* is represented by a tuple of four elements $(a,b,c,d) \in \text{Sym}(n)^4$
such that $abcd = 1$. A PTS is *1-cylinder* if every element of its braid group orbit has the property that $ab$ is an $n$-cycle. Our goal is to find non-cyclic 1-cylinder PTS, in other words, tuples such that 
* for every $g \in B_4$, if $g\cdot(a,b,c,d) = (a',b',c',d')$ then $a'b'$ is an $n$-cycle, and 
* $\langle a,b,c,d\rangle$ is a non-cyclic subgroup of $\text{Sym}(n)$.  

We're particularly interested if there are examples where the permutations $a,b,c,d$ do not include the identity permutation.

We use the class `Constellation` from the `sage::combinat` library. A *constellation* is a tuple of permutations in $\text{Sym(n)}$ whose product is the identity. The *length* of a constellation is the number of elements in the tuple,
and the *degree* is the $n$ of the symmetric group the permutations come from.
The `constellation` library by default only returns connected constellations, i.e. 
constellations whose permutations have a connected orbit when acting on 
$\{1,\dots,n\}$. In order to be a 1-cylinder surface, a PTS must be represented by a connected 
constellation, so this is ok.

In [282]:
from sage.all import SymmetricGroup
from sage.combinat import constellation

n = 5
Sn = SymmetricGroup(n)
all_pts = Constellations(4,n) # only returns connected constellations
horizontal_one_cyl_set = set()
for pts in all_pts:
    if (pts.g(0)*pts.g(1)).cycle_type() == [n]:
        horizontal_one_cyl_set.add(pts)
print(f'Total number of degree {n} PTS: {len(all_pts)}')
print(f'Total number of degree {n} PTS with a horizontal cyl: {len(horizontal_one_cyl_set)}')

Total number of degree 5 PTS: 1647384
Total number of degree 5 PTS with a horizontal cyl: 345600


In [ ]:
class PTS:
    # n is an integer > 1, constellation is an instance of the Constellation 
    # class on Sym(n)
    def __init__(self, n, constellation):
        self.n = n 
        self.constellation = constellation
        self.orbit = None

    def __eq__(self, other):
        return (self.constellation == other.constellation)
    
    def __neq__(self, other):
        return (self.constellation != other.constellation)

    def __hash__(self):
        return self.constellation.__hash__()

    def g(self,i):
        return self.constellation.g(i)
    
    def braid_group_orbit(self):
        orbit = set()
        waiting = [self]

        while waiting:
            
            c = waiting.pop()
            orbit.add(c)
            for i in range(3): # Constellation implements braid of 1st element and 4th element of tuple, which we want to skip, so only index with 0,1,2 here. TODO: ask Paul if this is correct
                cc = c.braid(i) # TODO: is this an error in the original library?
                if cc not in orbit:
                    # print(f'not in orbit')
                    waiting.append(cc)
        return orbit
    
    def iso_class_braid_group_orbit(self):
        orbit = []
        for t in self.constellation.braid_group_orbit().vertices():
            orbit.append(PTS(self.n, t))
        
        return orbit
    
    def braid(self, i):
        return PTS(self.n, self.constellation.braid_group_action(i))

    def is_cyclic(self):
        G = SymmetricGroup(self.n).subgroup([self.g(0), self.g(1), self.g(2), self.g(3)])
        return G.is_cyclic()
    
    def is_normal(self):
        G = SymmetricGroup(self.n).subgroup([self.g(0), self.g(1), self.g(2), self.g(3)])
        return G.is_normal() 
    
    # check if there is one cylinder in the horizontal direction
    def check_horizontal_direction(self):
        if (self.g(0)*self.g(1)).cycle_type() == [self.n]:
            return True 
        else:
            return False
    
    def is_one_cyl_surface(self):
        if self.orbit == None:
            self.braid_group_orbit()

        for t in self.orbit: # t not nec = pts, but will be isom. constellations
            if not PTS(self.n, t).check_horizontal_direction():
                return False
        return True
    
    def contains_identity(self):
        for i in range(0,4):
            if self.g(i).cycle_type() == ([1]*self.n):
                return True 
        return False

    def __str__(self):
        return f' ({self.g(0)},  {self.g(1)},  {self.g(2)},  {self.g(3)})'

    def verbose_print(self):
        print('PTS defined by:')
        print(f' a: {self.g(0)}')
        print(f' b: {self.g(1)}')
        print(f' c: {self.g(2)}')
        print(f' d: {self.g(3)}\n')
    
        print(f'cycle type: {self.constellation.profile()}')
        print(f'is cyclic: {self.is_cyclic()}')
        print(f'is normal: {self.is_normal()}')
        print(f'is 1-cylinder: {self.is_one_cyl_surface()}')
        print(f'Braid group orbit:')
        if self.orbit == None:
            self.orbit = self.braid_group_orbit()
        for t in self.orbit:
            print(t)
        print(f'\n')


In [330]:
orbit_reps = set()
orbit_computed = set()
computed = 0 # for tracking computation progress

total = len(horizontal_one_cyl_set)
print(f'0/{total}')
candidate_surfaces = [PTS(n,constellation) for constellation in horizontal_one_cyl_set]
for p in candidate_surfaces:
    if p in orbit_computed:
        computed +=1
        continue 
    else:
        orbit = p.braid_group_orbit()
        orbit_set= set(orbit)
        for o in orbit:
            orbit_computed.add(o)
        
        p.orbit = orbit
        orbit_reps.add(p)

        computed += 1
        print(f'{computed}/{total}')
print(f'{computed}/{total}')

0/345600
1/345600
2/345600
3/345600
4/345600
6/345600
7/345600
8/345600
9/345600
10/345600
11/345600
12/345600
13/345600
14/345600
15/345600
16/345600
17/345600
20/345600
21/345600
22/345600
23/345600
24/345600
25/345600
27/345600
28/345600
29/345600
30/345600
32/345600
34/345600
36/345600
38/345600
40/345600
41/345600
42/345600
43/345600
44/345600
45/345600
47/345600
51/345600
52/345600
53/345600
57/345600
59/345600
60/345600
62/345600
63/345600
66/345600
67/345600
68/345600
69/345600
73/345600
75/345600
76/345600
77/345600
87/345600
102/345600
107/345600
109/345600
110/345600
112/345600
121/345600
124/345600
129/345600
133/345600
134/345600
135/345600
142/345600
146/345600
147/345600
161/345600
163/345600
166/345600
170/345600
176/345600
181/345600
192/345600
196/345600
199/345600
201/345600
208/345600
217/345600
229/345600
232/345600
235/345600
249/345600
252/345600
268/345600
274/345600
288/345600
289/345600
296/345600
299/345600
319/345600
333/345600
341/345600
346/345600
378/3456

In [331]:
one_cyl_orbit_reps = set()

for pts in orbit_reps:
    if pts.is_one_cyl_surface():
        one_cyl_orbit_reps.add(pts)

print(len(one_cyl_orbit_reps))

50


In [332]:
one_cyl_orbit_reps_cycle_types = {}
for pts in one_cyl_orbit_reps:
    key = tuple(sorted(pts.constellation.profile()))
    if not one_cyl_orbit_reps_cycle_types.get(key)==None:
        one_cyl_orbit_reps_cycle_types.get(key).append(pts)
    else:
        one_cyl_orbit_reps_cycle_types.update({key: [pts]})
print(f'Number of cycle types: {len(one_cyl_orbit_reps_cycle_types.keys())}')
for key in one_cyl_orbit_reps_cycle_types.keys():
    print(f'{len(one_cyl_orbit_reps_cycle_types.get(key))} orbit reps of type {key}')

Number of cycle types: 2
26 orbit reps of type ([1, 1, 1, 1, 1], [5], [5], [5])
24 orbit reps of type ([5], [5], [5], [5])


In [ ]:
rep = list(one_cyl_orbit_reps)[21]
rep.verbose_print()

PTS defined by:
 a: (1,2,3,4,5)
 b: (1,2,5,3,4)
 c: ()
 d: (1,3,4,2,5)

cycle type: ([5], [5], [1, 1, 1, 1, 1], [5])
is cyclic: False
is normal: True
is 1-cylinder: True
Braid group orbit:
 ((1,4,5,2,3),  (),  (1,4,2,3,5),  (1,4,3,5,2))
 ((1,4,2,3,5),  (),  (1,4,3,5,2),  (1,4,5,2,3))
 ((),  (1,2,4,5,3),  (1,4,5,2,3),  (1,5,2,4,3))
 ((1,3,5,4,2),  (1,3,2,5,4),  (1,5,4,3,2),  ())
 ((1,5,4,3,2),  (1,5,2,4,3),  (),  (1,4,3,5,2))
 ((1,3,2,5,4),  (1,3,5,4,2),  (1,3,4,2,5),  ())
 ((1,3,5,4,2),  (1,4,2,3,5),  (),  (1,3,4,2,5))
 ((1,4,3,5,2),  (),  (1,5,4,3,2),  (1,5,2,4,3))
 ((),  (1,3,2,5,4),  (1,3,5,4,2),  (1,3,4,2,5))
 ((1,3,2,5,4),  (1,5,3,2,4),  (),  (1,5,4,3,2))
 ((1,2,4,5,3),  (1,5,2,4,3),  (1,5,3,2,4),  ())
 ((1,3,4,2,5),  (),  (1,3,5,4,2),  (1,4,2,3,5))
 ((),  (1,2,3,4,5),  (1,4,2,3,5),  (1,4,5,2,3))
 ((),  (1,5,4,3,2),  (1,4,3,5,2),  (1,3,5,4,2))
 ((1,3,4,2,5),  (1,2,3,4,5),  (1,2,5,3,4),  ())
 ((1,2,5,3,4),  (1,3,2,5,4),  (1,3,4,2,5),  ())
 ((1,5,4,3,2),  (),  (1,3,2,5,4),  (1,5,3,2

In [363]:
for (i,rep) in enumerate(one_cyl_orbit_reps):
    print(f'{i}: {rep.is_cyclic()}, {len(rep.orbit)}')
    if not rep.is_cyclic():
        print(rep)

0: True, 12
1: True, 4
2: True, 12
3: True, 4
4: True, 4
5: True, 4
6: True, 12
7: True, 12
8: True, 4
9: True, 12
10: True, 4
11: True, 12
12: True, 12
13: True, 12
14: True, 12
15: True, 12
16: True, 12
17: True, 4
18: True, 12
19: True, 4
20: True, 12
21: False, 240
 ((1,2,3,4,5),  (1,2,5,3,4),  (),  (1,3,4,2,5))
22: True, 12
23: True, 12
24: True, 12
25: True, 4
26: True, 4
27: True, 4
28: True, 4
29: True, 4
30: True, 4
31: True, 12
32: True, 4
33: True, 4
34: True, 12
35: False, 240
 ((1,5,2,3,4),  (1,3,4,5,2),  (1,5,3,4,2),  ())
36: True, 12
37: True, 4
38: True, 4
39: True, 4
40: True, 12
41: True, 12
42: True, 12
43: True, 4
44: True, 4
45: True, 4
46: True, 12
47: True, 4
48: True, 4
49: True, 12
